In [71]:
# importing pandas as pd
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import shutil

In [72]:
device_dir_paths = [
    'AwairAirQuality',
    'LiFXBulb',
    # 'PixStarPhotoFrame',
    # 'RingDoorBell',
    'TPLinkCamera',
    # 'TribySpeaker',
    # 'AmazonEcho/1',
    # 'AmazonEcho/2',
    # 'AmazonEcho/3',
    # 'AmazonEcho/4',
    # 'AmazonEcho/5',
    # 'AmazonEcho/6',
    # 'AmazonEcho/7',
    # 'AmazonEcho/8',
    # 'AmazonEcho/9',
    # 'NestProtect/1',
    # 'NestProtect/2',
    # 'NestProtect/3',
    # 'NestProtect/4',
    # 'NestProtect/5',
    # 'NestProtect/6',
    # 'NestProtect/7',
    # 'WithingsSleepSensor/1',
    # 'WithingsSleepSensor/2',
    # 'WithingsSleepSensor/3',
    # 'WithingsSleepSensor/4',
    # 'SamsungCamera',
    # 'WithingsBabyMonitor'
]

In [73]:
# for RESPONSE attribute files
# root_dir_path = '../../../data/attributes/V3/response_attributes'
# final_output_file = '../../../results/attributes/TLS/standard_nonstandard_response_attributes.csv'  

##################################################################################

# for REQUEST attribute files
root_dir_path = '../../../data/attributes/V3/request_attributes'
final_output_file = '../../../results/attributes/TLS/standard_nonstandard_request_attributes.csv'  

In [74]:
'''
data_device =    
    {
        device: {
            protocol: df of attributes.csv
            ...
        } ,
        ...
    }
'''

# Create an empty list to store the data from CSV files
data_device = {}

for device_dir_path in device_dir_paths:
    data_proto = {}
    directory_path  = os.path.join(root_dir_path, device_dir_path)
    # Get a list of all files in the directory
    file_list = os.listdir(directory_path)

    # Iterate over each file in the directory
    for file_name in file_list:
        # reading only the attributes files
        if 'tlsattributes' in file_name:
            proto = file_name.split('attributes')[0]  # tls
            # Check if the file is a CSV file
            if file_name.endswith('.csv'):
                # Create the file path
                file_path = os.path.join(directory_path, file_name)
                # print(f'device_dir_path: {device_dir_path}  |  proto: {proto}')
                # Read the CSV file and append its data to the list
                try:
                    # df = pd.read_csv(file_path, sep=',,,')
                    df = pd.read_csv(file_path, sep=',,,')
                except pd.errors.EmptyDataError:
                    df = pd.DataFrame()  # Create an empty DataFrame 

                # data.append(df)

                data_proto[proto] = df
                # data_proto[proto] = 'df'
    
    data_device[device_dir_path] = data_proto

# # Write dictionary as JSON
# with open('output.json', 'w') as json_file:
#     json.dump(data_device, json_file)

# data_device['AwairAirQuality']['tls'][' violations']






/tmp/ipykernel_12164/308301489.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=',,,')
/tmp/ipykernel_12164/308301489.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=',,,')
/tmp/ipykernel_12164/308301489.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=',,,')


In [75]:
for i in data_device['LiFXBulb']['tls'].columns:
    print(i)


srcMac
 dstMac
 ethType
 srcIp
 dstIp
 ipProto
 srcPort
 dstPort
client-hello-extensions
client-hello-cipher-suites
client-hello-version,,


In [76]:
'''
unique_attr_device =    
    {
        device: {
            protocol: {
                attr_name: {
                    standard:{}
                        value_attr: number_of_ccurences,
                        ...
                    },
                    non_standard:{}
                        value_attr: number_of_ccurences,
                        ...
                    }
                }
                ...
                'unique_flows_using_attribute': [list of unique flows]
                'num_violated_flows': len(unique_flows_using_attribute)
            },
            ...
        } ,
        ...
    }
'''

# srcMac, dstMac, ethType, srcIp, dstIp, ipProto, srcPort, dstPort

unique_attr_device = {}
for device, protos in data_device.items():  # per device
    print('device:', device)
    unique_attr_proto = {}
    for proto, df in protos.items():  # per protocol

        if True:  # proto == 'tls':  # running only for a selected protocol ######################################################################
            print(f'\t{device} --> proto: {proto}')
            attr = {}
            # num_violated_flows = 0

            for col in df.columns[8:]:
                
                if col == 'server-hello-certificates':  # skipping a specific attribute ######################################################################
                    continue
                if (device == 'AwairAirQuality' or device == 'LiFXBulb') and col == 'client-hello-extensions':
                    continue

                print(f'\t\t{device}:{proto} --> attr: {col}')
                unique_flows_using_attribute = []   
                # attr_name = col
                attr_name_dict_standardPort = {}
                attr_name_dict_NONstandardPort = {}
                non_standard_MUDFlows = {}
                for index in df.index:  # per line in the df
                    # print('\t\t\tindex:', index)
                    value_attr = df[col][index]  ####
                    # if value_attr == ' ':
                    # print(f'{device}:{proto}:{col} --> {value_attr}')
                    # making a flow signature
                    value_flow = df[' srcIp'][index] + '-' + df[' dstIp'][index] + '-' + str(df[' ipProto'][index]) + '-' + str(df[' srcPort'][index]) + '-' + str(df[' dstPort'][index])
                    
                    if df[' ipProto'][index] == 6 and (df[' srcPort'][index]==443 or df[' dstPort'][index]==443):
                        if value_attr not in attr_name_dict_standardPort:
                            attr_name_dict_standardPort[value_attr] = 0
                        attr_name_dict_standardPort[value_attr] += 1
                    else:  # non-standard port
                        if value_attr not in attr_name_dict_NONstandardPort:
                            attr_name_dict_NONstandardPort[value_attr] = 0
                        attr_name_dict_NONstandardPort[value_attr] += 1

                        # recording the non-standard flow used
                        flowSig = str(df[' ipProto'][index]) + '-' + str(df[' srcPort'][index]) + '-' + str(df[' dstPort'][index])
                        if flowSig not in non_standard_MUDFlows:
                            non_standard_MUDFlows[flowSig] = 0
                        non_standard_MUDFlows[flowSig] += 1                

                    if value_flow not in unique_flows_using_attribute:  # adding unique flow with a violation
                        unique_flows_using_attribute.append(value_flow)  
                        # num_violated_flows += 1
                # attr_name_dict['unique_flows_using_attribute'] = []  # unique_flows_using_attribute  # commented out because not needed at this point, can uncomment to be used later
                # attr_name_dict['number_of_unique_flows'] = len(unique_flows_using_attribute)
                attr[col] = {'standard': attr_name_dict_standardPort, 'nonstandard': attr_name_dict_NONstandardPort}

            attr['number_of_unique_flows'] = len(unique_flows_using_attribute)
            attr['nonstandard_flows'] = non_standard_MUDFlows
            unique_attr_proto[proto] = attr
            print(f'\t\t\tnumber of unique flows: ', len(unique_flows_using_attribute))

    # #####################################################################
    # df_to_save_temp = pd.DataFrame(unique_attr_proto)
    # # Specify the output CSV file path
    # output_file = '../../results/attributes/delete/'+device+'.csv'
    # # if os.path.exists(output_dir):
    # #     shutil.rmtree(output_dir)
    # # os.mkdir(output_dir)
    # # Write the DataFrame to the CSV file
    # df_to_save_temp.to_csv(df_to_save_temp, index=True, sep='|')
    # #####################################################################

    unique_attr_device[device] = unique_attr_proto

# with open('output2.json', 'w') as json_file:
#     json.dump(unique_viol_device, json_file)
# print(unique_attr_device)

device: AwairAirQuality
	AwairAirQuality --> proto: tls
		AwairAirQuality:tls --> attr: client-hello-cipher-suites
		AwairAirQuality:tls --> attr: client-hello-version,,
			number of unique flows:  826
device: LiFXBulb
	LiFXBulb --> proto: tls
		LiFXBulb:tls --> attr: client-hello-cipher-suites
		LiFXBulb:tls --> attr: client-hello-version,,
			number of unique flows:  131
device: TPLinkCamera
	TPLinkCamera --> proto: tls
		TPLinkCamera:tls --> attr: client-hello-extensions
		TPLinkCamera:tls --> attr: client-hello-cipher-suites
		TPLinkCamera:tls --> attr: client-hello-version,,
			number of unique flows:  26


In [77]:
# df_to_save = pd.DataFrame(unique_attr_device)
# df_to_save['TPLinkCamera']['classic-stun']

In [78]:
df_to_save = pd.DataFrame(unique_attr_device)

# Specify the output CSV file path
# final_output_file = '../../results/attributes/v1_request_attributes.csv'  # define in the beginning

# Write the DataFrame to the CSV file
df_to_save.to_csv(final_output_file, index=True, sep='|')

df_to_save.head()

,AwairAirQuality,LiFXBulb,TPLinkCamera
tls,{'client-hello-cipher-suites': {'standard': {'...,{'client-hello-cipher-suites': {'standard': {}...,"{'client-hello-extensions': {'standard': {}, '..."
